In [ ]:
import numpy as np
import time
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from dgl import model_zoo,DGLGraph
import math
import networkx as nx
import matplotlib.pyplot as plt
#from utils_mpnn import Meter, set_random_seed, collate, EarlyStopping, load_model,load_brl_dataset,regress,run_a_train_epoch,run_an_eval_epoch
import argparse
# from sklearn import svm, datasets
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import plot_confusion_matrix
import seaborn as sn
import pandas as pd
from sklearn import metrics
import collections
import torch.nn as nn
import torch.nn.functional as F
from utils_mpnn_classification import Meter

# define NN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        
        self.fc1 = nn.Linear(365*7+7, 4096)
        self.fc2 = nn.Linear(4096,4096)
        self.fc3 = nn.Linear(4096,4096)
        self.fc4_1 = nn.Linear(4096,120)
        self.fc4_2 = nn.Linear(4096,1)
        #self.droput = nn.Dropout(0.1)
        #self.sig=nn.Sigmoid()
        
    def forward(self,x):
        x = F.relu(self.fc1(x))
        #x = self.droput(x)
        x = F.relu(self.fc2(x))
        #x = self.droput(x)
        x = F.relu(self.fc3(x))
        #x = self.droput(x)
        x1 = self.fc4_1(x)
        x2 = self.fc4_2(x)#x2 = self.sig(self.fc4_2(x))
        return [x1,x2]#x
# initialize the NN
model = Net()

print(torch.cuda.current_device())
torch.cuda.set_device(1)
print(torch.cuda.current_device())

loss_fn_cn =nn.CrossEntropyLoss(reduction='none')
loss_fn_a =nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
#stopper = EarlyStopping(mode='lower', patience=500, filename='model_saved/kld/early_stop.pth')
args={};args['device']=torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(args['device'])

class MyDataset(Dataset):
    def __init__(self, data, target_cn, target_a,transform=None):
        self.data = torch.from_numpy(data).float()
        self.target_cn = torch.from_numpy(target_cn).long()
        self.target_a = torch.from_numpy(target_a).float()
        self.transform = transform
    def __getitem__(self, index):
        x = self.data[index]
        y_cn = self.target_cn[index]
        y_a = self.target_a[index]
        if self.transform:
            x = self.transform(x)
        return x, y_cn, y_a
    def __len__(self):
        return len(self.data)

data_path_x='data/data_mlp/';data_path_label='data/label_multitask/'
mode='train'
dataset = MyDataset(np.load(data_path_x+'features_'+mode+'.npz')['arr_0'], 
                    np.load(data_path_label+'c_label_'+mode+'.npz')['arr_0'],
                    np.load(data_path_label+'a_label_'+mode+'.npz')['arr_0'])
train_loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True,pin_memory=True)

mode='val'
dataset = MyDataset(np.load(data_path_x+'features_'+mode+'.npz')['arr_0'], 
                    np.load(data_path_label+'c_label_'+mode+'.npz')['arr_0'], 
                    np.load(data_path_label+'a_label_'+mode+'.npz')['arr_0'])
val_loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=False,pin_memory=True)

mode='test'
dataset = MyDataset(np.load(data_path_x+'features_'+mode+'.npz')['arr_0'],
                    np.load(data_path_label+'c_label_'+mode+'.npz')['arr_0'],
                    np.load(data_path_label+'a_label_'+mode+'.npz')['arr_0'])
test_loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=False,pin_memory=True)

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf  # set initial "min" to infinity

In [ ]:
#number of epochs to train the model
ep_list=[2000,4000,6000]
n_epochs = 3000
for epoch in range(n_epochs):
    epoch in ep_list:
        torch.save({'model_state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()}, 'model_saved/mlp_multitask/'+str(epoch)+'.pth')
        for param_group in optimizer.param_groups: print(param_group['lr'])
        for param_group in optimizer.param_groups: param_group['lr'] = param_group['lr']*0.1
        for param_group in optimizer.param_groups: print(param_group['lr'])
    # monitor losses
    train_loss = 0
    valid_loss = 0
    # train the model
    model.train() # prep model for training
    
    train_meter = Meter()
    for data,label_cn,label_a in train_loader:
        data = data.to(args['device'])
        label_cn = label_cn.to(args['device'])
        label_a = label_a.to(args['device'])
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        output_cn = output[0].to(args['device'])
        output_a = output[1].to(args['device'])
        # calculate the loss
        #loss = (loss_fn(output,label).float()).mean()
        loss_cn = (loss_fn_cn(output_cn, label_cn).float()).mean()
        loss_area = (loss_fn_a(output_a,label_a).float()).mean()
        loss=(loss_cn+loss_area)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        _, output_cn = torch.max(output_cn.data, 1)
        train_meter.update(output_cn, label_cn)#train_meter.update(output_cn, loss_cn)
        
    eval_meter_cn = Meter()
    eval_meter_a = Meter()
    # validate the model #
    model.eval()  # prep model for evaluation
    for data,label_cn,label_a in val_loader:
        data = data.to(args['device'])
        label_cn = label_cn.to(args['device'])
        label_a = label_a.to(args['device'])
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        output_cn = output[0].to(args['device'])
        output_a = output[1].to(args['device'])
        # calculate the loss
        loss_cn = (loss_fn_cn(output_cn, label_cn).float()).mean()
        loss_area = (loss_fn_a(output_a,label_a).float()).mean()
        loss=(loss_cn+10*loss_area)
        # update running validation loss
        _, output_cn = torch.max(output_cn.data, 1)
        eval_meter_cn.update(output_cn, label_cn)
        eval_meter_a.update(output_a, label_a)
    
    # print training/validation statistics # calculate average loss over an epoch
    train_loss=train_meter.compute_metric('acc')
    valid_loss_cn=eval_meter_cn.compute_metric('acc')
    valid_loss_a=np.mean(eval_meter_a.compute_metric('rmse'))
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tacc: {:.6f}\tValidation Loss: {:.6f}'.format(epoch+1, train_loss,valid_loss_cn,valid_loss_a))
    
    # save model if validation loss has decreased
    if valid_loss_cn <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss_cn))
        #torch.save(model.state_dict(), 'model_saved/mlp_cn/model.pt')
        torch.save({'model_state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()}, 'model_saved/mlp_multitask/model.pth')
        
        valid_loss_min = valid_loss_cn

In [ ]:
state=torch.load('model_saved/mlp_multitask/model.pth') 
model.load_state_dict(state['model_state_dict']) 
#optimizer.load_state_dict(state['optimizer'])

model.eval()
cn_p=[];cn_t=[];a_t=[];a_p=[]
with torch.no_grad():
    for data,label_cn,label_a in test_loader:
        data = data.to(args['device'])
        label_cn = label_cn.to(args['device'])
        label_a = label_a.to(args['device'])
        
        cn_t.append(label_cn.cpu().detach().numpy())
        a_t.append(label_a.cpu().detach().numpy())
        
        prediction = model(data)
        cn_p.append(prediction[0].cpu().detach().numpy())
        a_p.append(prediction[1].cpu().detach().numpy())

cn_p=np.concatenate(cn_p,axis=0)
cn_p=np.argmax(cn_p,axis=1)
a_p=np.concatenate(a_p,axis=0)
cn_t=np.concatenate(cn_t,axis=0)
a_t=np.concatenate(a_t,axis=0)



acc=np.sum(cn_p==cn_t)/len(cn_p)
print('acc: ',acc)

w_idx=np.where(cn_p!=cn_t)
c_idx=np.where(cn_p==cn_t)

area=np.load('data/label_multitask/a_label_test.npz')['arr_0']
print('acc',np.mean(area[w_idx]),np.mean(area[c_idx]))

import sklearn.metrics as metrics
t_area=a_t.copy();p_area=a_p.copy()
#plt.plot(p_area.flatten(),t_area.flatten(),'.',color='darkblue',alpha=0.2)
plt.plot(t_area[c_idx].flatten(),p_area[c_idx].flatten(),'.',color='aqua',alpha=0.3)
plt.plot(t_area[w_idx].flatten(),p_area[w_idx].flatten(),'.',color='red',alpha=0.3)
plt.ylim(np.min([a_p,a_t]), np.max([a_p,a_t]))
plt.xlim(np.min([a_p,a_t]), np.max([a_p,a_t]))
plt.grid()
plt.savefig('images/mlp_multitask/test_area.png', bbox_inches = 'tight')
plt.show()

y_t=a_t.copy();y_p=a_p.copy()
print(y_t.shape,y_p.shape)#(
import sklearn.metrics as metrics
mae = metrics.mean_absolute_error(y_t, y_p)
mse = metrics.mean_squared_error(y_t, y_p)
rmse=np.sqrt(mse)
print('mae', mae, '| rmse:',rmse)
print('actual mean', np.mean(y_t),'| pred mean',np.mean(y_p))
print(np.corrcoef(y_t.flatten(),y_p.flatten()))
print('rmse/range',rmse/(np.max(y_t)-np.min(y_t)))
print('mape',np.mean(np.abs(y_t-y_p)/y_t))
iqr= np.subtract(*np.percentile(y_t, [75, 25]))
print('rmse/iqr',rmse/iqr)
print('rmse/mean',rmse/np.mean(y_t))
print('actual min max', np.min(y_t),np.max(y_t))
print('pred min max', np.min(y_p),np.max(y_p))